## Poker Gameplay Engine


Importing required functions

In [1]:
import pandas as pd
#setting pandas options
pd.set_option('display.max_columns', None) #helps with testing to understand how data is added to the tracking dataframe

import numpy as  np
import random

At the start of each hand create a new deck. Shuffle the new deck and pop cards to the users in the game and the for the board cards.

##DataFrame to store hand Results

In [2]:
#colum names to add to the dataframe below
columns = [
    "hand_id",
    "user_hand",
    "com_hand",
    "user_chips",
    "com_chips",
    "winnings",
    "winner",
    "preflop_user_hand_strength",
    "preflop_com_hand_strength",
    "preflop_user_decision",
    "preflop_user_raise_amount",
    "preflop_computer_decision",
    "preflop_computer_raise_amount",
    "preflop_user_decision_2",
    "preflop_user_raise_amount_2",
    "preflop_computer_decision_2",
    "flop_cards",
    "postflop_user_hand_strength",
    "postflop_user_decsision",
    "postflop_user_bet",
    "postflop_user_raise_amount",
    "postflop_com_hand_strength",
    "postflop_computer_decision",
    "postflop_computer_bet",
    "postflop_computer_decision_2",
    "postflop_computer_re_raise_amount",
    "postflop_user_decision_2",
    "postflop_potsize",
    "turn_card",
    "turn_user_hand_strength",
    "turn_user_decision",
    "turn_user_bet",
    "turn_user_raise_amount",
    "turn_com_hand_strength",
    "turn_computer_decision",
    "turn_computer_bet",
    "turn_computer_decision_2",
    "turn_computer_re_raise_amount",
    "turn_user_decision_2",
    "post_turn_potsize",
    "river_card",
    "river_user_hand_strength",
    "river_user_decision",
    "river_user_bet",
    "river_user_raise_amount",
    "river_com_hand_strength",
    "river_computer_decision",
    "river_computer_bet",
    "river_computer_decision_2",
    "river_computer_re_raise_amount",
    "river_user_decision_2"
]

# Create the DataFrame with these columns
game_results_df = pd.DataFrame(columns=columns)

## Maintenence functions

In [3]:
#global board variable
board = []

def clear_board(board):
  """this function clears the board of cards"""
  board = []
  return board

def reset_stack(player_1, player_2, starting_stack):
  """
  this function resets the players chips to an equal starting stack of the users choice.
  """
  player_1.chips = starting_stack
  player_2.chips = starting_stack
  return player_1, player_2

def clear_dataframe(dataframe):
  game_results_df = pd.DataFrame(columns=columns)
  return game_results_df

def reset_game(player_1, player_2):
  global board
  global game_results_df
  clear_hand(player_1, player_2)
  board = clear_board(board)
  reset_stack(player_1, player_2, 100000)
  game_results_df = clear_dataframe(game_results_df)


##Preflop functions



In [4]:
#creating the poker deck
def create_deck():
  """
  this function creates a deck of cards and shuffles them
  """
  suits = ["hearts", "diamonds", "clubs", "spades"]
  ranks = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "jack", "queen", "king", "ace"]
  cards = [(rank, suit) for suit in suits for rank in ranks]
  random.shuffle(cards)
  return cards



In [5]:
#creating a bot and the user class
class Player:
   def __init__(self, name, chips):
    self.name = name
    self.chips = chips
    self.hand = [] #this will be filled as the cards are delt to the players

#names for bot BluffZero, PokerBot
com = Player("com", 1000)
user = Player("user", 1000)

players = [com, user]

In [6]:
def deal_cards(cards, player_1 = user, player_2 = com):
  """
  this function deals cards to the players
  """
  for i in range(2):
    player_1.hand.append(cards.pop())
    player_2.hand.append(cards.pop())

In [7]:
def clear_hand(player_1 = user, player_2 = com):
  """
  this function clears the hand of the players
  """
  player_1.hand = []
  player_2.hand = []

## Displaying cards functions

In [8]:
#updated cards to no longer be a global variable - need to go back and pass 'cards' everywhere in the pipeline.

def flop(cards):
  """
  Funtion to take three random cards from the deck and place them on the board.
  """
  global board
  for i in range(3):
    random_card = random.choice(cards)
    cards.remove(random_card)
    board.append(random_card)
  print(board)

def turn(cards):
  """
  This function takes a random card from the deck and places it on the board.
  """
  global board
  random_card = random.choice(cards)
  cards.remove(random_card)
  board.append(random_card)
  print(f"The turn is the {random_card} the board shows {board}")

def river(cards):
  """
  This function takes a random card from the deck and places it on the board.
  """
  global board
  random_card = random.choice(cards)
  cards.remove(random_card)
  board.append(random_card)
  print(f"The river is the {random_card} the board shows {board}")

## showdown winner functions

In [9]:
#determining winning hand

#get seven card hand
def get_seven_card_hand(player, board):
    return [card for card in player.hand + board]

#hand rankings
import itertools

def evaluate_hand(hand):
    rank_map = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6,
                '7': 7, '8': 8, '9': 9, '10': 10,
                'jack': 11, 'queen': 12, 'king': 13, 'ace': 14}

    ranks = sorted([rank_map[card[0]] for card in hand], reverse=True)
    suits = [card[1] for card in hand]
    rank_counts = {rank: ranks.count(rank) for rank in set(ranks)}

    is_flush = len(set(suits)) == 1
    ranks_asc = sorted(ranks)
    is_straight = ranks_asc == list(range(ranks_asc[0], ranks_asc[0] + 5))

    if is_straight and is_flush:
        return (9, max(ranks))  # Straight flush
    elif 4 in rank_counts.values():
        four = max(rank for rank, count in rank_counts.items() if count == 4)
        kicker = max(r for r in ranks if r != four)
        return (8, four, kicker)
    elif sorted(rank_counts.values()) == [2, 3] or sorted(rank_counts.values()) == [3, 2]:
        three = max(rank for rank, count in rank_counts.items() if count == 3)
        pair = max(rank for rank, count in rank_counts.items() if count == 2)
        return (7, three, pair)
    elif is_flush:
        return (6, *ranks)
    elif is_straight:
        return (5, max(ranks))
    elif 3 in rank_counts.values():
        three = max(rank for rank, count in rank_counts.items() if count == 3)
        kickers = [r for r in ranks if r != three]
        return (4, three, *kickers[:2])
    elif list(rank_counts.values()).count(2) == 2:
        pairs = sorted((rank for rank, count in rank_counts.items() if count == 2), reverse=True)
        kicker = max(rank for rank in ranks if rank not in pairs)
        return (3, *pairs, kicker)
    elif 2 in rank_counts.values():
        pair = max(rank for rank, count in rank_counts.items() if count == 2)
        kickers = [r for r in ranks if r != pair]
        return (2, pair, *kickers[:3])
    else:
        return (1, *ranks[:5])

def best_hand(seven_cards):
    all_combos = itertools.combinations(seven_cards, 5)
    return max(all_combos, key=evaluate_hand)

def determine_winner(players, board, pot, row_idx):
    results = []
    for player in players:
        seven = get_seven_card_hand(player, board)
        best = best_hand(seven)
        hand_rank = evaluate_hand(best)
        results.append((hand_rank, player.name, best))

    results.sort(reverse=True)  # higher ranks first
    winner = results[0]
    print(f"{winner[1]} wins with hand {winner[2]} ranked {winner[0]}")

    if winner[1] == "com":
      com.chips += pot
      game_results_df.at[row_idx, "winnings"] = pot
      game_results_df.at[row_idx, "winner"] = "com"

    else:
      user.chips += pot
      game_results_df.at[row_idx, "winnings"] = pot
      game_results_df.at[row_idx, "winner"] = "user"



## Hand strength at each street

In [10]:
#important functionality to get the hand strength of each player using a preflop hand strength chart

def preflop_hand_strength(card1, card2):


  preflop_strengths = {
    # Pocket pairs
    'AA': 1.000,
    'KK': 0.961,
    'QQ': 0.929,
    'JJ': 0.902,
    'TT': 0.878,
    '99': 0.854,
    '88': 0.828,
    '77': 0.800,
    '66': 0.771,
    '55': 0.742,
    '44': 0.713,
    '33': 0.683,
    '22': 0.653,

    # Suited Broadways
    'AKs': 0.820,
    'AQs': 0.787,
    'AJs': 0.753,
    'ATs': 0.719,
    'KQs': 0.727,
    'KJs': 0.694,
    'KTs': 0.660,
    'QJs': 0.679,
    'QTs': 0.646,
    'JTs': 0.613,

    # Offsuit Broadways
    'AKo': 0.797,
    'AQo': 0.765,
    'AJo': 0.731,
    'ATo': 0.697,
    'KQo': 0.706,
    'KJo': 0.672,
    'KTo': 0.639,
    'QJo': 0.657,
    'QTo': 0.623,
    'JTo': 0.590,

    # Suited Aces (non‐broadway)
    'A9s': 0.685,
    'A8s': 0.651,
    'A7s': 0.616,
    'A6s': 0.581,
    'A5s': 0.547,
    'A4s': 0.512,
    'A3s': 0.478,
    'A2s': 0.443,

    # Offsuit Aces (non‐broadway)
    'A9o': 0.660,
    'A8o': 0.626,
    'A7o': 0.591,
    'A6o': 0.556,
    'A5o': 0.522,
    'A4o': 0.487,
    'A3o': 0.453,
    'A2o': 0.418,

    # Suited Connectors and One‐Gappers
    'T9s': 0.632,
    '98s': 0.599,
    '87s': 0.564,
    '76s': 0.529,
    '65s': 0.494,
    '54s': 0.460,
    '43s': 0.425,
    '32s': 0.390,
    'J9s': 0.621,
    'K9s': 0.648,
    'Q9s': 0.608,
    'J8s': 0.587,
    'T8s': 0.555,
    '97s': 0.520,
    '86s': 0.486,
    '75s': 0.451,
    '64s': 0.417,
    '53s': 0.382,
    '42s': 0.348,

    # Offsuit Connectors and One‐Gappers
    'T9o': 0.607,
    '98o': 0.574,
    '87o': 0.539,
    '76o': 0.504,
    '65o': 0.469,
    '54o': 0.435,
    '43o': 0.400,
    '32o': 0.365,
    'J9o': 0.595,
    'K9o': 0.622,
    'Q9o': 0.582,
    'J8o': 0.560,
    'T8o': 0.528,
    '97o': 0.493,
    '86o': 0.459,
    '75o': 0.424,
    '64o': 0.389,
    '53o': 0.354,
    '42o': 0.320,

    # Suited Gappers 2 (two‐gappers)
    'J8s': 0.587,
    'T8s': 0.555,
    '97s': 0.520,
    '86s': 0.486,
    '75s': 0.451,
    '64s': 0.417,
    '53s': 0.382,
    '42s': 0.348,

    # Offsuit Gappers 2 (two‐gappers)
    'J8o': 0.560,
    'T8o': 0.528,
    '97o': 0.493,
    '86o': 0.459,
    '75o': 0.424,
    '64o': 0.389,
    '53o': 0.354,
    '42o': 0.320,

    # Suited Gappers 3
    'J7s': 0.514,
    'T7s': 0.480,
    '96s': 0.444,
    '85s': 0.410,
    '74s': 0.375,
    '63s': 0.340,
    '52s': 0.306,

    # Offsuit Gappers 3
    'J7o': 0.488,
    'T7o': 0.453,
    '96o': 0.417,
    '85o': 0.383,
    '74o': 0.348,
    '63o': 0.314,
    '52o': 0.280,

    # Suited 4‐gappers
    'J6s': 0.328,
    'T6s': 0.293,
    '95s': 0.259,
    '84s': 0.225,
    '73s': 0.191,
    '62s': 0.157,

    # Offsuit 4‐gappers
    'J6o': 0.301,
    'T6o': 0.266,
    '95o': 0.232,
    '84o': 0.197,
    '73o': 0.163,
    '62o': 0.129,

    # Suited 5‐gappers (example values)
    'J5s': 0.124,
    'T5s': 0.089,
    '94s': 0.054,
    '83s': 0.019,

    # Offsuit 5‐gappers (example values)
    'J5o': 0.098,
    'T5o': 0.064,
    '94o': 0.030,
    '83o': 0.010,

    # Remaining very weak hands
    '72o': 0.036,
    '62o': 0.031,
    '52o': 0.026,
    '43o': 0.022,
    '32o': 0.018
}



  rank_map = {
    '2': '2', '3': '3', '4': '4', '5': '5', '6': '6',
    '7': '7', '8': '8', '9': '9', '10': 'T',
    'jack': 'J', 'queen': 'Q', 'king': 'K', 'ace': 'A'
}

  rank1_raw, suit1 = card1
  rank2_raw, suit2 = card2

  rank1 = rank_map.get(rank1_raw.lower(), '?')
  rank2 = rank_map.get(rank2_raw.lower(), '?')

  if '?' in (rank1, rank2):
    raise ValueError(f"Invalid rank in card: {card1}, {card2}")

  # Pair
  if rank1 == rank2:
      key = rank1 * 2
  else:
      suited = suit1 == suit2
      # Sort ranks to match key ordering (e.g., AQ not QA)
      sorted_ranks = sorted([rank1, rank2], key=lambda r: '23456789TJQKA'.index(r), reverse=True)
      key = ''.join(sorted_ranks) + ('s' if suited else 'o')

  return preflop_strengths.get(key, 0.40)  # Default strength if not found

#heres the functionality for each steet besides preflop:
def get_hand_strength_at_street(player, board):
    """
    Returns the hand strength rank tuple at the current street
    based on the board state (flop, turn, or river).
    """
    current_hand = get_seven_card_hand(player, board)
    best = best_hand(current_hand)
    return evaluate_hand(best)

#converting the tuple score to a rank 1-10
def score_hand_strength(rank_tuple):
    """
    Converts a tuple like (6, 14, 13, 11, 9, 7) into a float score.
    Higher categories (9=straight flush, 1=high card) dominate.
    Kickers provide decimal detail.
    """
    category = rank_tuple[0]
    kickers = rank_tuple[1:]

    # Normalize kicker contribution (base-15 ensures no overlap across categories)
    decimal = 0
    for i, kicker in enumerate(kickers):
        decimal += kicker / (15 ** (i + 1))

    return round(category + decimal, 4)





##Gameflow pipeline

make sure pot is reset to zero after the dataframe is updated...

In [43]:
#this function includes everything that can happen preflop
def gameflow(player_1 = user, player_2 = com):
  """
  this function runs the preflop gameflow
  """

  global board
  board = []

  #for determining the winner need to loop through both players
  players = [player_1, player_2]

  #creating, shuffling, and dealing the cards to the players
  pot = 0
  cards = create_deck()
  deal_cards(cards, player_1 = user, player_2 = com)

  #setting row index for data logging
  row_idx = len(game_results_df)

  #setting a global small and big blind
  big_blind = 2
  small_blind = 1

  #automatically subtracting the blind amounts from the players stacks
  player_1.chips -= small_blind
  player_2.chips -= big_blind

  #adding the blind amount to the pot winnnings
  pot += small_blind + big_blind

  #showing the user thier hand and how many chips each player has remaining
  print(f"{player_1.name} has {player_1.chips} chips")
  print(f"your hand is {player_1.hand}")


  print(f"\n{player_2.name} has {player_2.chips} chips")
  print(f"the computer's hand is a mystery")

  print("\n")

  #adding inital characteristics to the dataframe for the hand
  game_results_df.loc[row_idx] = {
    "hand_id" : row_idx,
    "user_hand": player_1.hand,
    "com_hand": player_2.hand,
    "user_chips": player_1.chips,
    "com_chips": player_2.chips,
    "winnings": None,
    "winner" : None,
    # initialize all decision columns to None
    # preflop variables
    "preflop_user_hand_strength" : None,
    "preflop_com_hand_strength" : None,
    "preflop_user_decision": None,
    "preflop_user_raise_amount": None,
    "preflop_computer_decision": None,
    "preflop_computer_raise_amount": None,
    "preflop_user_decision_2": None,
    "preflop_user_raise_amount_2": None,
    "preflop_computer_decision_2": None,
    # postflop variables
    "flop_cards" : None,
    "postflop_user_hand_strength" : None,
    "postflop_com_hand_strength" : None,
    "postflop_user_decsision" : None,
    "postflop_user_bet" : None,
    "postflop_user_raise_amount" : None,
    "postflop_computer_decision" : None,
    "postflop_computer_bet" : None,
    "postflop_computer_decision_2" : None,
    "postflop_computer_re_raise_amount" : None,
    "postflop_user_decision_2" : None,
    "postflop_potsize" : None,
    # turn variables
    "turn_card" : None,
    "turn_user_hand_strength" : None,
    "turn_com_hand_strength" : None,
    "turn_user_decision" : None,
    "turn_user_bet" : None,
    "turn_user_raise_amount" : None,
    "turn_computer_decision" : None,
    "turn_computer_bet" : None,
    "turn_computer_decision_2" : None,
    "turn_computer_re_raise_amount" : None,
    "turn_user_decision_2" : None,
    "turn_potsize" : None,
    # river variables
    "river_card" : None,
    "river_user_hand_strength" : None,
    "river_com_hand_strength" : None,
    "river_user_decision" : None,
    "river_user_bet" : None,
    "river_user_raise_amount" : None,
    "river_computer_decision" : None,
    "river_computer_bet" : None,
    "river_computer_decision_2" : None,
    "river_computer_re_raise_amount" : None,
    "river_user_decision_2" : None,
    "river_potsize" : None
}

  #letting the user know its thier turn to act and the potential options
  print(f"{player_1.name} is the small blind and needs to match {big_blind}")

  #variable to determine if the hand is over yet or to continue to the next segment
  round_over = False

#-------------------------------------------------------------------------------------------------------------------#
#####################################################################################################################
#                      PRE FLOP DECISION MAKING                                                                    #
#####################################################################################################################
#-------------------------------------------------------------------------------------------------------------------#


  #give the user the option to fold, call, or raise. (user inital decision)
  user_decision = input("Do you want to fold, call, or raise?: ").lower()

  user_preflop_hand_strength = preflop_hand_strength(user.hand[0], user.hand[1])
  game_results_df.at[row_idx, "preflop_user_hand_strength"] = user_preflop_hand_strength

  com_preflop_hand_strength = preflop_hand_strength(com.hand[0], com.hand[1])
  game_results_df.at[row_idx, "preflop_com_hand_strength"] = com_preflop_hand_strength

  if user_decision == "fold":
    print("User folds")
    print(f"{player_2.name} wins {pot}")
    print("\nGet ready for the next hand...")
    player_2.chips += pot
    pot = 0
    clear_hand()
    #adding the results of the hand to the data frame
    game_results_df.at[row_idx, "preflop_user_decision"] = "fold"
    game_results_df.at[row_idx, "winnings"] = pot
    game_results_df.at[row_idx, "winner"] = player_2.name
    round_over = True
    clear_board(board)

  elif user_decision == "call":
    print("user calls")
    player_1.chips -= small_blind #essentially the user enters another small blind (1 more chip)
    pot += small_blind #adding the additional chip to the pot

    #adding the users preflop decision to the dataframe
    game_results_df.at[row_idx, "preflop_user_decision"] = "call"

    #now the computer gets to check or raise in reponse to the users decison to call and adds it to the dataframe
    com_decision = random.choice(["check", "raise"]) # (computer initial decision)
    game_results_df.at[row_idx, "preflop_computer_decision"] = com_decision

    if com_decision == "check":
      print("computer checks, here comes the flop...")
      flop(cards = cards)
      game_results_df.at[row_idx, "flop_cards"] = board #adds the flop cards to the dataframe
      game_results_df.at[row_idx, "postflop_potsize"] = pot
    else:
      computer_raise_amount = random.choice([pot * 2, pot * 3, pot * 5]) #giving the computer raise options (computer inital raise amount)
      print(f"The computer has decided to raise {computer_raise_amount}")
      game_results_df.at[row_idx, "preflop_computer_raise_amount"] = computer_raise_amount
      player_2.chips -= computer_raise_amount
      pot += computer_raise_amount

      #now the user gets to respond to the raise
      user_decision = input("do you want to fold, call, or raise? ") #user second decision
      game_results_df.at[row_idx, "preflop_user_decision_2"] = user_decision
      if user_decision == "fold":
        print("user folds")
        print(f"{player_2.name} wins {pot}")
        print("\nget ready for the next hand...")
        player_2.chips += pot
        clear_hand()
        game_results_df.at[row_idx, "winnings"] = pot
        game_results_df.at[row_idx, "winner"] = player_2.name
        round_over = True
        clear_board(board)
        pot = 0
      elif user_decision == "call":
        print("user calls")
        player_1.chips -= computer_raise_amount
        pot += computer_raise_amount
        print(f"The pot is {pot} here comes the flop...")
        flop(cards = cards)
        game_results_df.at[row_idx, "flop_cards"] = board #adds the flop cards to the dataframe
        game_results_df.at[row_idx, "postflop_potsize"] = pot
      else:
        user_re_raise_amount = int(input(f"you must raise at least 2x more than {computer_raise_amount}")) #user second raise amount
        game_results_df.at[row_idx, "preflop_user_raise_amount_2"] = user_re_raise_amount
        pot += user_re_raise_amount
        player_1.chips -= user_re_raise_amount

        #the computer now has the opportunity to respond to the re-raise with either a call or a fold
        com_decision = random.choice(["call", "fold"]) #computer second decision
        game_results_df.at[row_idx, "preflop_computer_decision_2"] = com_decision
        if com_decision == "call":
          print("computer calls")
          player_2.chips -= (user_re_raise_amount - computer_raise_amount)
          pot += (user_re_raise_amount - computer_raise_amount)
          print(f"The pot is {pot} here comes the flop...")
          flop(cards = cards)
          game_results_df.at[row_idx, "flop_cards"] = board #adds the flop cards to the dataframe
          game_results_df.at[row_idx, "postflop_potsize"] = pot
        else:
          print("computer folds")
          print(f"{player_1.name} wins {pot}")
          print("\nget ready for the next hand...")
          player_1.chips += pot
          clear_hand()
          game_results_df.at[row_idx, "winnings"] = pot
          game_results_df.at[row_idx, "winner"] = player_1.name
          round_over = True
          clear_board(board)
          pot = 0

  #if user decision is raise
  else:
    user_raise_amount = int(input(f"How much do you want to raise? You must raise to at least {big_blind * 2}: ")) #preflop_user_raise_amount
    game_results_df.at[row_idx, "preflop_user_raise_amount"] = user_raise_amount
    game_results_df.at[row_idx, "preflop_user_decision"] == "raise"
    pot += user_raise_amount
    player_1.chips -= user_raise_amount

    #now the computer has to respond to the raise
    com_decision = random.choice(["fold", "call", "raise"])
    game_results_df.at[row_idx, "preflop_computer_decision"] = com_decision

    if com_decision == "fold":
      print("computer folds")
      print(f"{player_1.name} wins {pot}")
      print("\nget ready for the next hand...")
      player_1.chips += pot
      clear_hand()
      game_results_df.at[row_idx, "winnings"] = pot
      game_results_df.at[row_idx, "winner"] = player_1.name
      round_over = True
      clear_board(board)
      pot = 0
    elif com_decision == "call":
      print("computer calls")
      player_2.chips -= (user_raise_amount - big_blind)
      pot += (user_raise_amount - big_blind)
      print(f"The pot is {pot} here comes the flop...")
      flop(cards = cards)
      game_results_df.at[row_idx, "flop_cards"] = board
      game_results_df.at[row_idx, "postflop_potsize"] = pot
    else:
      #gives the computer the option to reraise
      computer_raise_amount = random.choice([pot * 2, pot * 3, pot * 5])
      print(f"The computer has decided to raise {computer_raise_amount}")
      game_results_df.at[row_idx, "preflop_computer_raise_amount"] = computer_raise_amount
      player_2.chips -= computer_raise_amount

      #after the computer raises the re-raises the user can either call or fold

      user_decision = input("do you want to fold or call? ")
      game_results_df.at[row_idx, "preflop_user_decision_2"] = user_decision

      if user_decision == "fold":
        print("user folds")
        print(f"{player_2.name} wins {pot}")
        print("\nget ready for the next hand...")
        player_2.chips += pot
        clear_hand()
        game_results_df.at[row_idx, "winnings"] = pot
        game_results_df.at[row_idx, "winner"] = player_2.name
        round_over = True
        clear_board(board)
        pot = 0
      #user call the com raise
      else:
        print("user calls")
        player_1.chips -= computer_raise_amount
        pot += computer_raise_amount
        print(f"The pot is {pot} here comes the flop...")
        flop(cards = cards)
        game_results_df.at[row_idx, "flop_cards"] = board
        game_results_df.at[row_idx, "postflop_potsize"] = pot

#-------------------------------------------------------------------------------------------------------------------#
#####################################################################################################################
#                      POST FLOP DECISION MAKING                                                                    #
#####################################################################################################################
#-------------------------------------------------------------------------------------------------------------------#

  if round_over == False:

    hand_strength_user = get_hand_strength_at_street(player = player_1, board = board)
    numerical_hand_strength_user = score_hand_strength(hand_strength_user)
    game_results_df.at[row_idx, "postflop_user_hand_strength"] = numerical_hand_strength_user

    hand_strength_com = get_hand_strength_at_street(player = player_2, board = board)
    numerical_hand_strength_com = score_hand_strength(hand_strength_com)
    game_results_df.at[row_idx, "postflop_com_hand_strength"] = numerical_hand_strength_com

    #the action is on the user to make the first decision postflop
    user_decision = input("Do you want to check or bet?: ").lower()
    game_results_df.at[row_idx, "postflop_user_decision"] = user_decision

    if user_decision == "check":
      print("user checks")
      #now the computer gets the decision to check or bet
      com_decision = random.choice(["check", "bet"])
      game_results_df.at[row_idx, "postflop_computer_decision"] = com_decision

      if com_decision == "check":
        print("computer checks")
        print("here comes the turn...")
        turn(cards = cards) #prints to the user the 3 cards for the flop + the turn
        game_results_df.at[row_idx, "turn_card"] = board[3] #adds the flop cards to the dataframe
        game_results_df.at[row_idx, "postturn_potsize"] = pot

      #if the computer chooses to bet
      else:
        computer_bet_amount = random.choice([pot * 0.3, pot*0.5, pot*0.75, pot, pot*2])
        print(f"The computer has decided to bet {computer_bet_amount}")
        game_results_df.at[row_idx, "postflop_computer_bet"] = computer_bet_amount
        player_2.chips -= computer_bet_amount

        #now the user gets to respond with a call, fold, or raise
        user_decision = input("Do you want to call, fold, or raise?: ").lower()
        game_results_df.at[row_idx, "postflop_user_decision_2"] = user_decision

        if user_decision == "fold":
          print("user folds")
          print(f"{player_2.name} wins {pot}")
          print("\nget ready for the next hand...")
          player_2.chips += pot
          clear_hand()
          game_results_df.at[row_idx, "winnings"] = pot
          game_results_df.at[row_idx, "winner"] = player_2.name
          round_over = True
          clear_board(board)
          pot = 0

        elif user_decision == "call":
          print("user calls")
          player_1.chips -= computer_bet_amount
          pot += computer_bet_amount
          print(f"The pot is {pot} here comes the turn...")
          turn(cards = cards) #prints to the user the 3 cards for the flop + the turn
          game_results_df.at[row_idx, "turn_card"] = board[3] #adds the flop cards to the dataframe
          game_results_df.at[row_idx, "postturn_potsize"] = pot

        #user decision is raise
        else:
          user_raise_amount = int(input(f"you must raise to at least {computer_bet_amount * 2}"))
          game_results_df.at[row_idx, "postflop_user_raise_amount"] = user_raise_amount
          pot += user_raise_amount
          player_1.chips -= user_raise_amount

          #now the computer needs to respond to the users raise
          com_decision = random.choice(["fold", "call", "raise"])
          game_results_df.at[row_idx, "postflop_computer_decision_2"] = com_decision

          if com_decision == "fold":
            print("computer folds")
            print(f"{player_1.name} wins {pot}")
            print("\nget ready for the next hand...")
            player_1.chips += pot
            clear_hand()
            game_results_df.at[row_idx, "winnings"] = pot
            game_results_df.at[row_idx, "winner"] = player_1.name
            round_over = True
            clear_board(board)
            pot = 0
          elif com_decision == "call":
            print("computer calls")
            player_2.chips -= (user_raise_amount - computer_bet_amount)
            pot += (user_raise_amount - computer_bet_amount)
            print(f"The pot is {pot} here comes the turn...")
            turn(cards = cards) #prints to the user the 3 cards for the flop + the turn
            game_results_df.at[row_idx, "turn_card"] = board[3] #adds the flop cards to the dataframe
            game_results_df.at[row_idx, "postturn_potsize"] = pot

          #if the com decides to re_raise
          else:
            computer_re_raise_amount = random.choice([user_raise_amount * 0.5, user_raise_amount * 2, pot])
            print(f"The computer has decided to raise {computer_re_raise_amount}")
            game_results_df.at[row_idx, "postflop_computer_re_raise_amount"] = computer_re_raise_amount
            player_2.chips -= computer_re_raise_amount

            #now the user can make a decision in response to the re-raise
            user_decision = input("Do you want to call, fold?: ").lower()
            game_results_df.at[row_idx, "postflop_user_decision_2"] = user_decision

            if user_decision == "fold":
              print("user folds")
              print(f"{player_2.name} wins {pot}")
              print("\nget ready for the next hand...")
              player_2.chips += pot
              clear_hand()
              game_results_df.at[row_idx, "winnings"] = pot
              game_results_df.at[row_idx, "winner"] = player_2.name
              round_over = True
              clear_board(board)
              pot = 0

            #user calls the com raises
            else:
              print("user calls")
              player_1.chips -= (computer_re_raise_amount - user_raise_amount)
              pot += (computer_re_raise_amount - user_raise_amount)
              print(f"The pot is {pot} here comes the turn...")
              turn(cards = cards) #prints to the user the 3 cards for the flop + the turn
              game_results_df.at[row_idx, "turn_card"] = board[3] #adds the flop cards to the dataframe
              game_results_df.at[row_idx, "postturn_potsize"] = pot

    #users leads with a bet
    else:
      user_bet_amount = int(input(f"you must bet at least {big_blind * 2}: "))
      game_results_df.at[row_idx, "postflop_user_bet"] = user_bet_amount
      pot += user_bet_amount
      player_1.chips -= user_bet_amount

      #now the computer has to respond to the users bet amount
      com_decision = random.choice(["fold", "call", "raise"])
      game_results_df.at[row_idx, "postflop_computer_decision"] = com_decision

      if com_decision == "fold":
        print("computer folds")
        print(f"{player_1.name} wins {pot}")
        print("\nget ready for the next hand...")
        player_1.chips += pot
        clear_hand()
        game_results_df.at[row_idx, "winnings"] = pot
        game_results_df.at[row_idx, "winner"] = player_1.name
        round_over = True
        clear_board(board)
        pot = 0

      elif com_decision == "call":
        print("computer calls")
        player_2.chips -= user_bet_amount
        pot += user_bet_amount
        print(f"The pot is {pot} here comes the turn...")
        turn(cards = cards) #prints to the user the 3 cards for the flop + the turn
        game_results_df.at[row_idx, "turn_card"] = board[3] #adds the flop cards to the dataframe
        game_results_df.at[row_idx, "postturn_potsize"] = pot

      #the computer decides to raise the users bet
      else:
        computer_re_raise_amount = random.choice([user_bet_amount * 0.5, user_bet_amount * 2, pot])
        print(f"The computer has decided to raise {computer_re_raise_amount}")
        game_results_df.at[row_idx, "postflop_computer_re_raise_amount"] = computer_re_raise_amount
        player_2.chips -= computer_re_raise_amount

        #now the user can make a decision in response to the re-raise
        user_decision = input("Do you want to call, fold?: ").lower()
        game_results_df.at[row_idx, "postflop_user_decision_2"] = user_decision

        if user_decision == "fold":
          print("user folds")
          print(f"{player_2.name} wins {pot}")
          print("\nget ready for the next hand...")
          player_2.chips += pot
          clear_hand()
          game_results_df.at[row_idx, "winnings"] = pot
          game_results_df.at[row_idx, "winner"] = player_2.name
          round_over = True
          clear_board(board)
          pot = 0

        #user calls the com raises
        else:
          print("user calls")
          player_1.chips -= (computer_re_raise_amount - user_bet_amount)
          pot += (computer_re_raise_amount - user_bet_amount)
          print(f"The pot is {pot} here comes the turn...")
          turn(cards = cards) #prints to the user the 3 cards for the flop + the turn
          game_results_df.at[row_idx, "turn_card"] = board[3] #adds the flop cards to the dataframe
          game_results_df.at[row_idx, "postturn_potsize"] = pot

#-------------------------------------------------------------------------------------------------------------------#
#####################################################################################################################
#                      POST TURN DECISION MAKING                                                                    #
#####################################################################################################################
#-------------------------------------------------------------------------------------------------------------------#

  if round_over == False:

    hand_strength_user = get_hand_strength_at_street(player = player_1, board = board)
    numerical_hand_strength_user = score_hand_strength(hand_strength_user)
    game_results_df.at[row_idx, "turn_user_hand_strength"] = numerical_hand_strength_user

    hand_strength_com = get_hand_strength_at_street(player = player_2, board = board)
    numerical_hand_strength_com = score_hand_strength(hand_strength_com)
    game_results_df.at[row_idx, "turn_com_hand_strength"] = numerical_hand_strength_com

    #the action is on the user to make the first decision on the turn
    user_decision = input("Do you want to check or bet?: ").lower()
    game_results_df.at[row_idx, "turn_user_decision"] = user_decision

    if user_decision == "check":
      print("user checks")
      #now the computer gets the decision to check or bet
      com_decision = random.choice(["check", "bet"])
      game_results_df.at[row_idx, "turn_computer_decision"] = com_decision

      if com_decision == "check":
        print("computer checks")
        print("here comes the river...")
        river(cards = cards) #prints to the user the 3 cards for the flop + the turn
        game_results_df.at[row_idx, "river_card"] = board[4] #adds the flop cards to the dataframe
        game_results_df.at[row_idx, "postriver_potsize"] = pot

      #if the computer chooses to bet
      else:
        computer_bet_amount = random.choice([pot * 0.3, pot*0.5, pot*0.75, pot, pot*2])
        print(f"The computer has decided to bet {computer_bet_amount}")
        game_results_df.at[row_idx, "turn_computer_bet"] = computer_bet_amount
        player_2.chips -= computer_bet_amount

        #now the user gets to respond with a call, fold, or raise
        user_decision = input("Do you want to call, fold, or raise?: ").lower()
        game_results_df.at[row_idx, "turn_user_decision_2"] = user_decision

        if user_decision == "fold":
          print("user folds")
          print(f"{player_2.name} wins {pot}")
          print("\nget ready for the next hand...")
          player_2.chips += pot
          clear_hand()
          game_results_df.at[row_idx, "winnings"] = pot
          game_results_df.at[row_idx, "winner"] = player_2.name
          round_over = True
          clear_board(board)
          pot = 0

        elif user_decision == "call":
          print("user calls")
          player_1.chips -= computer_bet_amount
          pot += computer_bet_amount
          print(f"The pot is {pot} here comes the river...")
          river(cards = cards) #prints to the user the 3 cards for the flop + the turn
          game_results_df.at[row_idx, "river_card"] = board[4] #adds the flop cards to the dataframe
          game_results_df.at[row_idx, "postriver_potsize"] = pot

        #user decision is raise
        else:
          user_raise_amount = int(input(f"you must raise to at least {computer_bet_amount * 2}"))
          game_results_df.at[row_idx, "turn_user_raise_amount"] = user_raise_amount
          pot += user_raise_amount
          player_1.chips -= user_raise_amount

          #now the computer needs to respond to the users raise
          com_decision = random.choice(["fold", "call", "raise"])
          game_results_df.at[row_idx, "turn_computer_decision_2"] = com_decision

          if com_decision == "fold":
            print("computer folds")
            print(f"{player_1.name} wins {pot}")
            print("\nget ready for the next hand...")
            player_1.chips += pot
            clear_hand()
            game_results_df.at[row_idx, "winnings"] = pot
            game_results_df.at[row_idx, "winner"] = player_1.name
            round_over = True
            clear_board(board)
            pot = 0

          elif com_decision == "call":
            print("computer calls")
            player_2.chips -= (user_raise_amount - computer_bet_amount)
            pot += (user_raise_amount - computer_bet_amount)
            print(f"The pot is {pot} here comes the river...")
            river(cards = cards) #prints to the user the 3 cards for the flop + the turn
            game_results_df.at[row_idx, "river_card"] = board[4] #adds the flop cards to the dataframe
            game_results_df.at[row_idx, "postriver_potsize"] = pot

          #if the com decides to re_raise
          else:
            computer_re_raise_amount = random.choice([user_raise_amount * 0.5, user_raise_amount * 2, pot])
            print(f"The computer has decided to raise {computer_re_raise_amount}")
            game_results_df.at[row_idx, "turn_computer_re_raise_amount"] = computer_re_raise_amount
            player_2.chips -= computer_re_raise_amount

            #now the user can make a decision in response to the re-raise
            user_decision = input("Do you want to call, fold?: ").lower()
            game_results_df.at[row_idx, "turn_user_decision_2"] = user_decision

            if user_decision == "fold":
              print("user folds")
              print(f"{player_2.name} wins {pot}")
              print("\nget ready for the next hand...")
              player_2.chips += pot
              clear_hand()
              game_results_df.at[row_idx, "winnings"] = pot
              game_results_df.at[row_idx, "winner"] = player_2.name
              round_over = True
              clear_board(board)
              pot = 0

            #user calls the com raises
            else:
              print("user calls")
              player_1.chips -= (computer_re_raise_amount - user_raise_amount)
              pot += (computer_re_raise_amount - user_raise_amount)
              print(f"The pot is {pot} here comes the river...")
              river(cards = cards) #prints to the user the 3 cards for the flop + the turn
              game_results_df.at[row_idx, "river_card"] = board[4] #adds the flop cards to the dataframe
              game_results_df.at[row_idx, "postriver_potsize"] = pot

    #users leads with a bet
    else:
      user_bet_amount = int(input(f"you must bet at least {big_blind * 2}: "))
      game_results_df.at[row_idx, "turn_user_bet"] = user_bet_amount
      pot += user_bet_amount
      player_1.chips -= user_bet_amount

      #now the computer has to respond to the users bet amount
      com_decision = random.choice(["fold", "call", "raise"])
      game_results_df.at[row_idx, "turn_computer_decision"] = com_decision

      if com_decision == "fold":
        print("computer folds")
        print(f"{player_1.name} wins {pot}")
        print("\nget ready for the next hand...")
        player_1.chips += pot
        clear_hand()
        game_results_df.at[row_idx, "winnings"] = pot
        game_results_df.at[row_idx, "winner"] = player_1.name
        round_over = True
        clear_board(board)
        pot = 0

      elif com_decision == "call":
        print("computer calls")
        player_2.chips -= user_bet_amount
        pot += user_bet_amount
        print(f"The pot is {pot} here comes the river...")
        river(cards = cards) #prints to the user the 3 cards for the flop + the turn
        game_results_df.at[row_idx, "river_card"] = board[4] #adds the flop cards to the dataframe
        game_results_df.at[row_idx, "postriver_potsize"] = pot

      #the computer decides to raise the users bet
      else:
        computer_raise_amount = random.choice([user_raise_amount * 0.5, user_raise_amount * 2, pot])
        print(f"The computer has decided to raise {computer_raise_amount}")
        game_results_df.at[row_idx, "turn_computer_re_raise_amount"] = computer_raise_amount
        player_2.chips -= computer_raise_amount
        pot += computer_raise_amount

        #now the user can make a decision in response to the re-raise
        user_decision = input("Do you want to call, fold?: ").lower()
        game_results_df.at[row_idx, "turn_user_decision_2"] = user_decision

        if user_decision == "fold":
          print("user folds")
          print(f"{player_2.name} wins {pot}")
          print("\nget ready for the next hand...")
          player_2.chips += pot
          clear_hand()
          game_results_df.at[row_idx, "winnings"] = pot
          game_results_df.at[row_idx, "winner"] = player_2.name
          round_over = True
          clear_board(board)
          pot = 0
        #user calls the com raises
        else:
          print("user calls")
          player_1.chips -= (computer_raise_amount - user_bet_amount)
          pot += (computer_raise_amount - user_bet_amount)
          print(f"The pot is {pot} here comes the river...")
          river(cards = cards) #prints to the user the 3 cards for the flop + the turn
          game_results_df.at[row_idx, "river_card"] = board[4] #adds the flop cards to the dataframe
          game_results_df.at[row_idx, "postriver_potsize"] = pot


#-------------------------------------------------------------------------------------------------------------------#
#####################################################################################################################
#                      POST RIVER DECISION MAKING                                                                   #
#####################################################################################################################
#-------------------------------------------------------------------------------------------------------------------#

  if round_over == False:

    hand_strength_user = get_hand_strength_at_street(player = player_1, board = board)
    numerical_hand_strength_user = score_hand_strength(hand_strength_user)
    game_results_df.at[row_idx, "river_user_hand_strength"] = numerical_hand_strength_user

    hand_strength_com = get_hand_strength_at_street(player = player_2, board = board)
    numerical_hand_strength_com = score_hand_strength(hand_strength_com)
    game_results_df.at[row_idx, "river_com_hand_strength"] = numerical_hand_strength_com

    #the action is on the user to make the first decision on the turn
    user_decision = input("Do you want to check or bet?: ").lower()
    game_results_df.at[row_idx, "river_user_decision"] = user_decision

    if user_decision == "check":
      print("user checks")
      #now the computer gets the decision to check or bet
      com_decision = random.choice(["check", "bet"])
      game_results_df.at[row_idx, "river_computer_decision"] = com_decision

      if com_decision == "check":
        print("computer checks")
        print("here comes the showdown...")

        determine_winner(players = players, board= board, pot = pot, row_idx = row_idx)
        clear_board(board)

      #if the computer chooses to bet
      else:
        computer_bet_amount = random.choice([pot * 0.2, pot*0.4, pot*0.6, pot, pot*1.75])
        print(f"The computer has decided to bet {computer_bet_amount}")
        game_results_df.at[row_idx, "river_computer_bet"] = computer_bet_amount
        player_2.chips -= computer_bet_amount

        #now the user gets to respond with a call, fold, or raise
        user_decision = input("Do you want to call, fold, or raise?: ").lower()
        game_results_df.at[row_idx, "river_user_decision_2"] = user_decision

        if user_decision == "fold":
          print("user folds")
          print(f"{player_2.name} wins {pot}")
          print("\nget ready for the next hand...")
          player_2.chips += pot
          clear_hand()
          game_results_df.at[row_idx, "winnings"] = pot
          game_results_df.at[row_idx, "winner"] = player_2.name
          clear_board(board)
          pot = 0
        elif user_decision == "call":
          print("user calls")
          player_1.chips -= computer_bet_amount
          pot += computer_bet_amount
          print(f"The pot is {pot} here comes the showdown...")

          determine_winner(players = players, board= board, pot = pot, row_idx = row_idx)
          clear_board(board)
          clear_hand()

        #user decision is raise
        else:
          user_raise_amount = int(input(f"you must raise to at least {computer_bet_amount * 2}"))
          game_results_df.at[row_idx, "river_user_raise_amount"] = user_raise_amount
          pot += user_raise_amount
          player_1.chips -= user_raise_amount

          #now the computer needs to respond to the users raise
          com_decision = random.choice(["fold", "call", "raise"])
          game_results_df.at[row_idx, "river_computer_decision_2"] = com_decision

          if com_decision == "fold":
            print("computer folds")
            print(f"{player_1.name} wins {pot}")
            print("\nget ready for the next hand...")
            player_1.chips += pot
            clear_hand()
            game_results_df.at[row_idx, "winnings"] = pot
            game_results_df.at[row_idx, "winner"] = player_1.name
            clear_board(board)
            pot = 0
          elif com_decision == "call":
            print("computer calls")
            player_2.chips -= (user_raise_amount - computer_bet_amount)
            pot += (user_raise_amount - computer_bet_amount)
            print(f"The pot is {pot} here comes the showdown...")

            determine_winner(players = players, board= board, pot = pot, row_idx = row_idx)
            clear_board(board)
            clear_hand()

          #if the com decides to re_raise
          else:
            computer_re_raise_amount = random.choice([user_raise_amount * 0.5, user_raise_amount * 2, pot])
            print(f"The computer has decided to raise {computer_re_raise_amount}")
            game_results_df.at[row_idx, "river_computer_re_raise_amount"] = computer_re_raise_amount
            player_2.chips -= computer_re_raise_amount

            #now the user can make a decision in response to the re-raise
            user_decision = input("Do you want to call, fold?: ").lower()
            game_results_df.at[row_idx, "river_user_decision_2"] = user_decision

            if user_decision == "fold":
              print("user folds")
              print(f"{player_2.name} wins {pot}")
              print("\nget ready for the next hand...")
              player_2.chips += pot
              clear_hand()
              game_results_df.at[row_idx, "winnings"] = pot
              game_results_df.at[row_idx, "winner"] = player_2.name
              clear_board(board)
              pot = 0
            #user calls the com raises
            else:
              print("user calls")
              player_1.chips -= (computer_re_raise_amount - user_raise_amount)
              pot += (computer_re_raise_amount - user_raise_amount)
              print(f"The pot is {pot} here comes the showdown...")

              determine_winner(players = players, board= board, pot = pot, row_idx = row_idx)
              clear_board(board)
              clear_hand()

    #users leads with a bet
    else:
      user_bet_amount = int(input(f"you must bet at least {big_blind * 2}: "))
      game_results_df.at[row_idx, "river_user_bet"] = user_bet_amount
      pot += user_bet_amount
      player_1.chips -= user_bet_amount

      #now the computer has to respond to the users bet amount
      com_decision = random.choice(["fold", "call", "raise"])
      game_results_df.at[row_idx, "river_computer_decision"] = com_decision

      if com_decision == "fold":
        print("computer folds")
        print(f"{player_1.name} wins {pot}")
        print("\nget ready for the next hand...")
        player_1.chips += pot
        clear_hand()
        game_results_df.at[row_idx, "winnings"] = pot
        game_results_df.at[row_idx, "winner"] = player_1.name
        clear_board(board)
        pot = 0
      elif com_decision == "call":
        print("computer calls")
        player_2.chips -= user_bet_amount
        pot += user_bet_amount
        print(f"The pot is {pot} here comes the showdown...")

        determine_winner(players = players, board= board, pot = pot, row_idx=row_idx)
        clear_board(board)
        clear_hand()

      #the computer decides to raise the users bet
      else:
        computer_raise_amount = random.choice([user_bet_amount * 0.5, user_bet_amount * 2, pot])
        print(f"The computer has decided to raise {computer_raise_amount}")
        game_results_df.at[row_idx, "river_computer_re_raise_amount"] = computer_raise_amount
        player_2.chips -= computer_raise_amount
        pot += computer_re_raise_amount

        #now the user can make a decision in response to the re-raise
        user_decision = input("Do you want to call, fold?: ").lower()
        game_results_df.at[row_idx, "river_user_decision_2"] = user_decision

        if user_decision == "fold":
          print("user folds")
          print(f"{player_2.name} wins {pot}")
          print("\nget ready for the next hand...")
          player_2.chips += pot
          clear_hand()
          game_results_df.at[row_idx, "winnings"] = pot
          game_results_df.at[row_idx, "winner"] = player_2.name
          clear_board(board)
          pot = 0
        #user calls the com raises
        else:
          print("user calls")
          player_1.chips -= (computer_raise_amount - user_bet_amount)
          pot += (computer_raise_amount - user_bet_amount)
          print(f"The pot is {pot} here comes the showdown...")

          determine_winner(players = players, board= board, pot = pot, row_idx=row_idx)
          clear_board(board)
          clear_hand()



# GitHub requests  ☁

workflow to update notebook inside github repository

In [68]:
#connecting colab notebook to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [69]:
#github account details
!git config --global user.email "jasonmarchese2002@outlook.com"
!git config --global user.name "jmarchese1"

In [70]:
#first clone the repo no token needed because its public
!git clone https://github.com/jmarchese1/poker-ai-headsup.git

Cloning into 'poker-ai-headsup'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 3), reused 7 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (16/16), 39.52 KiB | 1.58 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [71]:
#creating directory
%cd /content/poker-ai-headsup/

/content/poker-ai-headsup


In [ ]:
#moving the notebook im working on in google drive into the cloned repository in colab
!cp "/content/drive/My Drive/Colab Notebooks/poker_bot.ipynb" '/content/ML_Poker/poker_bot.ipynb'

In [73]:
#check whats inside folder
!ls /content/poker-ai-headsup//

poker_bot.ipynb  README.md


In [74]:
#set the remote -- token was banned before for getting leaked so needed to create a new token, make sure to not leak it and delete this after running
"""
!git remote set-url origin \
  https://token@github.com/jmarchese1/ML_Poker.git
"""

In [77]:
#stage commit push
!git add core_engine.ipynb
!git commit -m "Your commit message"
!git push origin main

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date
